In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# ===============================
# 1. Load dataset
# ===============================
def load_data(directory, img_size=(64,64)):
    X = []
    y = []
    valid_extensions = (".jpg", ".jpeg", ".png")  # Only accept image files
    for label, folder in enumerate(["cats", "dogs"]):
        folder_path = os.path.join(directory, folder)
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if not file.lower().endswith(valid_extensions):
                continue  # skip non-image files
            try:
                img = image.load_img(file_path, target_size=img_size)
                img_array = image.img_to_array(img)
                img_array = img_array / 255.0  # Normalize
                X.append(img_array.flatten())
                y.append(label)
            except Exception as e:
                print("Skipping", file_path, ":", e)
    return np.array(X), np.array(y)

print("Loading training data...")
X_train, y_train = load_data(r"C:\Users\DSATM\Downloads\training_set\training_set")

print("Loading test data...")
X_test, y_test = load_data(r"C:\Users\DSATM\Downloads\test_set\test_set")

# ===============================
# 2. PCA
# ===============================
print("Applying PCA...")
pca = PCA(n_components=200)  # reduce dimensions
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# ===============================
# 3. Train SVM
# ===============================
print("Training SVM...")
svm = SVC(kernel="rbf", C=10, gamma=0.001)
svm.fit(X_train_pca, y_train)

# ===============================
# 4. Evaluate
# ===============================
print("Evaluating...")
y_pred = svm.predict(X_test_pca)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["Cat", "Dog"]))

# ===============================
# 5. Predict a single image
# ===============================
def predict_image(img_path, pca, svm_model, img_size=(64,64)):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = img_array.flatten().reshape(1, -1)

    img_pca = pca.transform(img_array)
    prediction = svm_model.predict(img_pca)
    return "Dog" if prediction[0] == 1 else "Cat"

# Example usage
print("\nSingle Image Prediction:")
print(predict_image(r"C:\Users\DSATM\Downloads\training_set\training_set\dogs\dog.801.jpg", pca, svm))
print(predict_image(r"C:\Users\DSATM\Downloads\training_set\training_set\cats\cat.8.jpg", pca, svm))
print(predict_image(r"C:\Users\DSATM\Downloads\training_set\training_set\cats\cat.89.jpg", pca, svm))
print(predict_image(r"C:\Users\DSATM\Downloads\training_set\training_set\dogs\dog.809.jpg", pca, svm))
print(predict_image(r"C:\Users\DSATM\Downloads\training_set\training_set\cats\cat.91.jpg", pca, svm))
print(predict_image(r"C:\Users\DSATM\Downloads\training_set\training_set\dogs\dog.836.jpg", pca, svm))



Loading training data...
Loading test data...
Applying PCA...
Training SVM...
Evaluating...
Accuracy: 0.6796836381611469
              precision    recall  f1-score   support

         Cat       0.68      0.69      0.68      1011
         Dog       0.68      0.67      0.68      1012

    accuracy                           0.68      2023
   macro avg       0.68      0.68      0.68      2023
weighted avg       0.68      0.68      0.68      2023


Single Image Prediction:
Dog
Cat
Cat
Dog
Cat
Dog
